In [3]:
import os, sys
import glob
import numpy as np
import pandas as pd
import subprocess
from praatio import textgrid as tgio
import argparse

sys.path.append('../../utils/')

from config import *
import dataset_utils as utils
from preproc_utils import cut_stimulus_segments

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [2]:
experiment_version = 'final-multimodal-01'
task = 'wheretheressmoke'
stim_type = 'video'
subject = 'sub-00001'

# set directories
stim_dir = os.path.join(BASE_DIR, 'stimuli')
presentation_orders_dir = os.path.join(stim_dir, 'presentation_orders', experiment_version, task)

if stim_type == 'video':
    stim_fn = glob.glob(os.path.join(stim_dir, 'video', f'{task}*.mp4'))
    jspsych_out_dir = os.path.join(presentation_orders_dir, 'jspsych-video')
else:
    stim_fn = glob.glob(os.path.join(stim_dir, 'audio', f'{task}*.wav'))
    jspsych_out_dir = os.path.join(presentation_orders_dir, 'jspsych')

NameError: name 'BASE_DIR' is not defined

In [ ]:

# load preprocessed transcript and find indices that are to be predicted
df_preproc_fn = os.path.join(presentation_orders_dir, 'preproc', f'{subject}_task-{task}.csv')
df_preproc = pd.read_csv(df_preproc_fn)

In [ ]:


	# Create sequential pairs
	candidate_idxs = np.where(df_preproc['NWP_Candidate'].to_numpy())[0] # First get indices
	candidate_idxs = np.concatenate([[0], candidate_idxs], axis=0) # Add the first item for the first cut
	segments = np.vstack((candidate_idxs[:-1], candidate_idxs[1:]-1)).T # Stack and make pairs
	segment_indices = segments.tolist()
	
	# perform audio segmenting for the current subject
	out_fns, df_segments = cut_stimulus_segments(df_preproc, task=p.task, stim_fn=stim_fn, stim_out_dir=stim_out_dir, segment_indices=segment_indices, stim_type=p.stim_type)
	df_participant = create_participant_df(df_preproc, df_segments)

	# now write the file for jspsych to use
	jspsych_fn = os.path.join(jspsych_out_dir, f'{p.subject}_task-{p.task}')
	df_participant.to_csv(f'{jspsych_fn}.csv', index=False)
	df_participant.to_json(f'{jspsych_fn}.json', orient='records')